In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [3]:
pd.set_option('display.max_row', 10)
df = pd.read_csv('../website_classification.csv')
df

,website_url,cleaned_website_text,Category
0,https://www.fubo.tv/lp/StreamTV/?irad=343747&i...,stream live tv sports fubotv free trial sport ...,Streaming Services
1,https://www.hulu.com/,stream tv movie live online hulu watch tv show...,Streaming Services
2,https://tv.youtube.com/,youtube tv watch dvr live sports shows news st...,Streaming Services
3,https://www.attwatchtv.com/,watchtv stream channel live tv want watchtv ex...,Streaming Services
4,https://www.klowdtv.com/,live tv streaming klowdtv klowdtv provide live...,Streaming Services
...,...,...,...
1134,https://bitmixing.net/,bitcoin mixer tumbler bitcoin blenderlooking f...,Computers and Technology
1135,https://finance.yahoo.com/news/bitcoin-mixer-e...,bitcoin mixer experts introduce top tumblers t...,Computers and Technology
1136,https://ransomwhe.re/,ransomwhereransomwhere is the open crowdsource...,Computers and Technology
1137,https://altenens.is/,altenen trust and safetyaltenen is a forum ded...,Computers and Technology


In [4]:
computers = pd.read_csv('../Hierarchal model/Computer/computer_subcategory_classification.csv')
computers

,website_url,cleaned_website_text,Category
0,http://www.kasrl.org/jaffe.html,japanese female facial expression jaffe datase...,Computers and Technology
1,https://www.gmdhshell.com/,best predictive analytic software free academi...,Computers and Technology
2,http://en.wikipedia.org/wiki/Neural_network,neural network wikipedia neural network conten...,Computers and Technology
3,http://cadingandcoding.blogspot.com/,cading coding cading coding tuesday march auto...,Computers and Technology
4,http://www.panzercad.com/,welcome bluehost rate web host provider free c...,Computers and Technology
...,...,...,...
304,https://bitmixing.net/,bitcoin mixer tumbler bitcoin blenderlooking f...,Cryptocurrency
305,https://finance.yahoo.com/news/bitcoin-mixer-e...,bitcoin mixer experts introduce top tumblers t...,Cryptocurrency
306,https://ransomwhe.re/,ransomwhereransomwhere is the open crowdsource...,Cyber Security
307,https://altenens.is/,altenen trust and safetyaltenen is a forum ded...,Cyber Security


In [5]:
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os


class ScrapTool:   
    def visit_url(self,website_url):
        '''
        Visit URL. Download the Content. Initialize the beautifulsoup object. Call parsing methods. Return Series object.
        '''
        #Set up Selenium webdriver
        PATH = 'C:/Users/shrey/Downloads/chromedriver_win32 (1)/chromedriver.exe'

        # Set the Chrome webdriver options
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        # Initialize the Chrome webdriver and pass the Chrome options
        service = Service(PATH)
        driver = webdriver.Chrome(service = service, options = chrome_options)

        #Load website
        driver.get(website_url)
        
        # Check if the cookie consent button is present
        cookie_button = None
        try:
            cookie_button = WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "cookie-consent-button")))
        except:
            pass

        # Handle cookies if the button is present
        if cookie_button:
            cookie_button.click()

        # Extract HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content,'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup)+self.get_html_meta_tags(soup)+self.get_html_heading_tags(soup)+
                                    self.get_text_content(soup)
        }
        
        #Convert to Series object and return
        return pd.Series(result)
    
    def get_website_name(self,website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])
    
    def get_html_title_tag(self,soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)
    
    def get_html_meta_tags(self,soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name=="meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords','description']]
        return ' '.join(content)
    
    def get_html_heading_tags(self,soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1","h2","h3","h4","h5","h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)
    
    def get_text_content(self,soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]',"h1","h2","h3","h4","h5","h6","noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore\
                and isinstance(tag, bs4.element.Comment)==False\
                and not stripped_tag.isnumeric()\
                and len(stripped_tag)>0:
                result.append(stripped_tag)
        return ' '.join(result)

In [6]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+',' ',text)
    return text

In [11]:
def content_generation(website):
    try:
        scrapTool = ScrapTool()
        web = scrapTool.visit_url(website)
        text = cleaning_text(web['website_text'])
        return text
    except Exception as e:
        print('Error: ',website)
        print(e)
        return None

In [12]:
websites = [
    "https://soufiane.website/ransomwatcher/ransomvictims.xml"
]


for website in websites:
    text = content_generation(website)
    if text is not None:
        print(text)
        # web = [website,text,'Computers and Technology']
        # df.loc[len(df)] = web
        # comp = [website,text,'Cyber Security']
        # computers.loc[len(computers)] = comp

Error:  https://soufiane.website/ransomwatcher/ransomvictims.xml
'NoneType' object has no attribute 'contents'


In [9]:
computers['Category'].value_counts()

Cyber Security              122
Computers and Technology     94
Cryptocurrency               93
Name: Category, dtype: int64

In [10]:
df['Category'].value_counts()

Computers and Technology           303
Social Networking and Messaging    114
Business/Corporate                 107
Streaming Services                 103
Sports                             100
                                  ... 
Games                               99
News                                93
Law and Government                  83
Adult                               21
Forums                              16
Name: Category, Length: 11, dtype: int64

In [42]:
computers.drop(222,inplace=True)

In [44]:
computers.reset_index(drop=True,inplace=True)

In [45]:
df.to_csv("website_classification.csv",index=False)
computers.to_csv("../Hierarchal model/Computer/computer_subcategory_classification.csv",index=False)